In [24]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [25]:
df_rating = pd.read_csv("C:/Users/91995/Downloads/archive/ml-100k/u.data", sep="\t")
df_rating.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df_rating = df_rating.drop(['timestamp'], axis=1)
df_rating2 = df_rating.groupby('user_id').filter(lambda x: len(x) >= 270)


df_user = pd.read_csv('C:/Users/91995/Downloads/archive/ml-100k/u.user', sep='|', header=None)
df_user.columns = ['user_id', 'age', 'gender', 'occupation', 'zip_code']
from sklearn.preprocessing import LabelEncoder
en = LabelEncoder()
df_user['gender'] = en.fit_transform(df_user['gender'])


item_cols = ['movie_id', 'movie_title', 'release_date', 'video_release_date', 'imdb_url']
genre_cols = ['unknown', 'action', 'adventure', 'animation', 'children', 'comedy', 'crime', 'documentary',
              'drama','fantasy', 'film_noir', 'horror', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 
              'war','western']

df_item_genre = pd.read_csv('C:/Users/91995/Downloads/archive/ml-100k/u.item', sep='|',encoding='latin-1')
df_item_genre.columns = item_cols + genre_cols


df_item = df_item_genre[item_cols]
use_item_cols = ['movie_id', 'movie_title', 'release_date']
df_item = df_item[use_item_cols]

df_genre = df_item_genre[['movie_id'] + genre_cols]

df3 = pd.merge(df_rating2, df_item, on='movie_id')
df3 = pd.merge(df3, df_genre, on='movie_id')
df3 = pd.merge(df3, df_user, on='user_id')


links = pd.read_csv("C:/Users/91995/Downloads/archive/ml-100k/links.csv")
links = links.dropna()


movies = pd.read_csv("C:/Users/91995/Downloads/archive/ml-100k/movies.csv")
movies.columns = ['movie_id2', 'movie_title', 'genres']


df3 = df3.join(movies.set_index('movie_title'), on='movie_title')

df3 = df3.drop(['occupation', 'zip_code', 'genres'], axis=1)


df3 = df3.dropna()

df3 = df3.drop(['unknown', 'documentary', 'fantasy', 'film_noir', 'war', 'western'], axis=1)
df3 = df3[df3['movie_id'].isin(links['movieId'].values)]

df_cold = df3[['user_id','animation','comedy','crime', 'drama', 'musical', 'mystery', 'romance', 'sci_fi', 'thriller', 'age', 'gender','rating']]
df_cold = df_cold.set_index('user_id')

matrix_id = df3.pivot_table(index='user_id', columns='movie_id', values='rating')

In [26]:
data_cold = df_cold.values

x = data_cold[:, :-1]
y = data_cold[:, -1]
print('x',x)
print('y',y)

from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import accuracy_score

bc = GradientBoostingClassifier()
bc.fit(x_train, y_train)
y_pred4 = bc.predict(x_test)

mae = mean_absolute_error(y_test, y_pred4)
print(mae)

x [[ 0  1  0 ...  0 27  1]
 [ 0  1  0 ...  0 27  1]
 [ 0  0  0 ...  1 27  1]
 ...
 [ 0  0  0 ...  0 35  0]
 [ 0  0  0 ...  0 35  0]
 [ 0  0  0 ...  0 35  0]]
y [5 2 3 ... 3 5 3]
0.854911631730374


In [27]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler


df_cold3 =df_cold.copy()

feature = df_cold3[['age']]

# Initialize the scaler
scaler = MinMaxScaler()

# Fit and transform the feature using the scaler
scaled_feature = scaler.fit_transform(feature)

print(scaled_feature)

df_cold3['age'] = scaled_feature

print(df_cold3)

[[0.29787234]
 [0.29787234]
 [0.29787234]
 ...
 [0.46808511]
 [0.46808511]
 [0.46808511]]
         animation  comedy  crime  drama  musical  mystery  romance  sci_fi  \
user_id                                                                       
286              0       1      0      0        0        0        0       0   
286              0       1      0      0        0        0        1       0   
286              0       0      0      0        0        0        0       0   
286              0       0      0      0        0        0        0       0   
286              0       0      0      1        0        0        0       0   
...            ...     ...    ...    ...      ...      ...      ...     ...   
18               0       0      0      0        0        0        0       1   
18               0       0      0      1        0        0        0       0   
18               0       0      0      1        0        0        0       0   
18               0       0      0      1 

In [28]:
data_cold3 = df_cold3.values
x = data_cold3[:, :-1]
y = data_cold3[:, -1]
print('x',x)
print('y',y)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=1)

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score

bc2 = GradientBoostingClassifier()
bc2.fit(x_train, y_train)
y_pred5 = bc2.predict(x_test)

mae = mean_absolute_error(y_test, y_pred5)
print(mae)

x [[0.         1.         0.         ... 0.         0.29787234 1.        ]
 [0.         1.         0.         ... 0.         0.29787234 1.        ]
 [0.         0.         0.         ... 1.         0.29787234 1.        ]
 ...
 [0.         0.         0.         ... 0.         0.46808511 0.        ]
 [0.         0.         0.         ... 0.         0.46808511 0.        ]
 [0.         0.         0.         ... 0.         0.46808511 0.        ]]
y [5. 2. 3. ... 3. 5. 3.]
0.854911631730374


In [29]:
#x_test is array of preferences of user
#eg: x_test = [1, 1, 1, 0, 0, 1, 0, 0, 1, 26, 1]
from sklearn.metrics.pairwise import cosine_similarity
import math

x_test = [1, 1, 1, 0, 0, 1, 0, 0, 1, 26, 1]
x_test = np.array([x_test])
y_pred = bc.predict(x_test)
new_user_array = np.append(x_test[0], y_pred)
new_user_array = np.array([new_user_array])
print(new_user_array)

rows = []
for row in range(len(df_cold)):
    ind = df_cold.index[row]
    sim = cosine_similarity(np.array(df_cold.iloc[row, :]).reshape(1, 12), new_user_array)[0][0]
    rows.append([ind, sim])

rows = sorted(rows, key = lambda x: x[1], reverse = True)

n = 5
similar_users = []
temp = []
for r in rows:
    if len(similar_users) == n:
        break

    if r not in similar_users:
        similar_users.append(r)
        temp.append(r[0])

print(similar_users)

similar_users = pd.DataFrame(similar_users, columns=['user_id', 'similarity'])
similar_users = similar_users.set_index('user_id')
print(similar_users)

similar_user_movies = matrix_id[matrix_id.index.isin(temp)].dropna(axis=1, how='all')

movie_score = {}
for i in similar_user_movies.columns:
    movie_rating = similar_user_movies[i]
    total = 0
    count = 0
    for u in similar_users.index:
        if math.isnan(movie_rating[u]) is False:
            score = similar_users.loc[u].item() *  movie_rating[u]
            total += score
            count += 1

    movie_score[i] = total/count

movie_score = pd.DataFrame(movie_score.items(), columns=['movie_id', 'score'])
ranked_movie_score = movie_score.sort_values(by='score', ascending=False)

[[ 1  1  1  0  0  1  0  0  1 26  1  4]]
[[758, 0.9992633389804575], [334, 0.9987646775507729], [94, 0.9985663075064698], [178, 0.9985663075064698], [286, 0.9985471521597764]]
         similarity
user_id            
758        0.999263
334        0.998765
94         0.998566
178        0.998566
286        0.998547


In [30]:
m = 16
temp = []

for m in ranked_movie_score['movie_id']:
    if len(temp) ==  16:
        break
    tmdb_id = links[links['movieId']==m].tmdbId.item()
#     if Watched.objects.filter(user=new_user, movie=tmdb_id).exists():
#         print('already watched', tmdb_id)
#         continue
    #skip movie if already watched 
    temp.append(m)

#tmdb movie id    
recommendations = []
for m in temp:
    if len(links[links['movieId']==m])== 0:
        print('hi')
        continue
    recommendations.append(links[links['movieId']==m].tmdbId.item())

print(recommendations)

[11331.0, 90214.0, 89333.0, 87078.0, 19101.0, 108365.0, 34996.0, 11107.0, 11224.0, 17585.0, 10714.0, 10331.0, 217802.0, 25739.0, 67365.0, 40480.0]
